[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=maplibre/3d_choropleth.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/maplibre/3d_choropleth.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

**Create a 3D choropleth map of Europe with countries extruded**

This source code of this example is adapted from the MapTiler SDK JS example - [Create a 3D choropleth map of Europe with countries extruded](https://docs.maptiler.com/sdk-js/examples/fill-extrusion).

Uncomment the following line to install [leafmap](https://leafmap.org) if needed.

In [1]:
# %pip install "leafmap[maplibre]"

import os
import urllib.request, json 
import pandas as pd

In [2]:
import leafmap.maplibregl as leafmap

To run this notebook, you will need an [API key](https://docs.maptiler.com/cloud/api/authentication-key/) from [MapTiler](https://www.maptiler.com/cloud/). Once you have the API key, you can uncomment the following code block and replace `YOUR_API_KEY` with your actual API key. Then, run the code block code to set the API key as an environment variable.

In [7]:
# Load income data
INCOME_URL = "https://www.irs.gov/pub/irs-soi/21incyallagi.csv"
PD_INCOME = pd.read_csv(INCOME_URL, encoding='ISO-8859-1')
PD_GROUPS = pd.read_csv(INCOME_URL, encoding='ISO-8859-1')

GROUPED = PD_GROUPS.groupby("COUNTYFIPS", as_index=False)['N1'].agg('sum').rename(columns={"N1": "total_taxpayers"})
print(GROUPED)
print(PD_INCOME)
MERGED = PD_INCOME.merge(PD_GROUPS, on="COUNTYFIPS", how="inner")

 

#GET HELP MERGING BY COUNTY ID
COUNTIES_URL = "https://open.gishub.org/data/us/us_counties.geojson"
with urllib.request.urlopen(COUNTIES_URL) as response:
    COUNTIES_JSON = json.load(response)
    PD_COUNTIES = pd.json_normalize(COUNTIES_JSON['features'])

# Add GeoJSON source
source = {
    "type": "geojson",
    "data": COUNTIES_URL,
}

     COUNTYFIPS  total_taxpayers
0             0      158700080.0
1             1        3389510.0
2             3        4592210.0
3             5        3419930.0
4             6          10080.0
..          ...              ...
325         800          45790.0
326         810         222450.0
327         820          11600.0
328         830           5860.0
329         840          13950.0

[330 rows x 2 columns]
       STATEFIPS STATE  COUNTYFIPS     COUNTYNAME  agi_stub        N1  \
0              1    AL           0        Alabama         1   52940.0   
1              1    AL           0        Alabama         2  255300.0   
2              1    AL           0        Alabama         3  465120.0   
3              1    AL           0        Alabama         4  524150.0   
4              1    AL           0        Alabama         5  291610.0   
...          ...   ...         ...            ...       ...       ...   
25547         56    WY          45  Weston County         4     640.0

In [4]:


# Create a map
m = leafmap.Map(center=[-100, 40], zoom=3, pitch=60, style="simple")


m.add_source('counties', source)

# Define the fill-extrusion layer
layer = {
    "id": "us-counties",
    "source": "counties",
    "type": "fill-extrusion",
    "paint": {
        "fill-extrusion-color": [
            "interpolate",
            ["linear"],
            ["get", "CENSUSAREA"],
            23.0, "#ffffff",
            200, "#fee6ce",
            500, "#fdd0a2",
            1000, "#8c2d04",
        ],
        "fill-extrusion-opacity": 1,
        "fill-extrusion-height": ["*", ["get", "CENSUSAREA"], 100],
    },
}

# Add the layer to the map
m.add_layer(layer)

# # Define the names layer
# names_layer = {
#     "id": "us-counties-name",
#     "source": "counties",
#     "type": "symbol",
#     "layout": {
#         "text-field": ["get", "NAME"],  # Assuming 'NAME' is the field for county names
#         'text-variable-anchor': ['top', 'bottom', 'left', 'right'],
#         'text-radial-offset': 0.5,
#         'text-justify': 'auto',
#     }
# }

# # Add the names layer to the map
# m.add_layer(names_layer)

# Add layer control
m.add_layer_control()

# Display the map
m

Failed to retrieve the MapTiler style. Defaulting to OpenFreeMap 'liberty' style.


Map(height='600px', map_options={'bearing': 0, 'center': (-100, 40), 'pitch': 60, 'style': 'https://tiles.open…